# Mouting Drive

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install tensorflow

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, Input, Flatten, MaxPooling1D, SpatialDropout1D, Activation

from keras.callbacks import EarlyStopping

from numpy import array
from sklearn.metrics import classification_report

import gensim
from gensim import models
from gensim.models import Word2Vec
import fasttext.util
import pandas as pd
import numpy as np

# Loading dataset

In [ ]:
train_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/train.csv")
val_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/dev.csv")
test_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/test.csv")

In [ ]:
print(f'train: {train_dataset.shape}\nval: {val_dataset.shape}\ntest: {test_dataset.shape}')

In [ ]:
train_dataset['label'].value_counts()

In [ ]:
train_x = train_dataset['text']
train_y = train_dataset['label']

val_x = val_dataset['text']
val_y = val_dataset['label']

test_x = test_dataset['text']
test_y = test_dataset['label']

# Embedding Setup

In [ ]:
tokenizer=Tokenizer(oov_token = "<OOV>", split=' ') # Splitting text based on whitespace and adding "Out of vocabulary"
tokenizer.fit_on_texts(train_x) # Using the tokenizer on out train dataset to tokenize the train dataset
train_encoded=tokenizer.texts_to_sequences(train_x)
# print(train_encoded)

In [ ]:
train_padded= pad_sequences(train_encoded, padding='post', maxlen=256)
# print(train_padded)

In [ ]:
train_padded.shape[1]

In [ ]:
# padding df_test
test_encoded=tokenizer.texts_to_sequences(test_x)
test_padded= pad_sequences(test_encoded, padding='post', maxlen=train_padded.shape[1])

In [ ]:
# padding df_validation
val_encoded=tokenizer.texts_to_sequences(val_x)
val_padded= pad_sequences(val_encoded, padding='post', maxlen=train_padded.shape[1])

# Loading FastText

In [ ]:
# import fasttext
import numpy as np

# # Load the FastText model
# fasttext_model = fasttext.load_model("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/notebooks/Tariq/fasttext/model_bn_300.bin")

# # Function that takes word vector as input and returns an embedding matrix
# def embedding_creation(EMBEDDING_DIM, word_vectors, tokenizer):
#     vocabulary_size = len(tokenizer.word_index) + 1
#     word_index = tokenizer.word_index
#     embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

#     for word, i in word_index.items():
#         try:
#             embedding_vector = word_vectors.get_word_vector(word)
#             embedding_matrix[i] = embedding_vector
#         except KeyError:
#             embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)

#     return embedding_matrix

# EMBEDDING_DIM = 300

# # Replace 'tokenizer' with the tokenizer object you have previously created
# # Assuming 'tokenizer' is defined earlier in your code

# embedding_matrix = embedding_creation(EMBEDDING_DIM, fasttext_model, tokenizer)


# Model Configuration

In [ ]:
!pip install scikit-learn

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have your text data in 'train_data', 'val_data', and 'test_data'

# Initialize a CountVectorizer to convert text data into BoW representation
vectorizer = CountVectorizer(max_features=5000)  # You can adjust the 'max_features' parameter as needed

# Fit and transform the training data
train_features = vectorizer.fit_transform(train_data)
train_y = train_labels  # Assuming you have labels for the training data

# Initialize and train the SVM classifier
svm_classifier = SVC(kernel='rbf', C=1.0, decision_function_shape='ovr', random_state=42)
svm_classifier.fit(train_features, train_y)

# Transform the validation and test data using the same vectorizer
val_features = vectorizer.transform(val_data)
test_features = vectorizer.transform(test_data)

# Make predictions on the validation set
val_predictions = svm_classifier.predict(val_features)

# Evaluate the model on the validation set
val_accuracy = accuracy_score(val_y, val_predictions)
val_classification_report = classification_report(val_y, val_predictions)

print(f"Validation Accuracy: {val_accuracy}")
print("Validation Classification Report:\n", val_classification_report)

# Make predictions on the test set
test_predictions = svm_classifier.predict(test_features)

# Evaluate the model on the test set
test_accuracy = accuracy_score(test_y, test_predictions)
test_classification_report = classification_report(test_y, test_predictions)

print(f"Test Accuracy: {test_accuracy}")
print("Test Classification Report:\n", test_classification_report)


In [ ]:
!pip install joblib

In [ ]:
import joblib
import os

# Save the trained model
model_filename = 'svm_model.pkl'
# Define the directory where you want to save the model
save_directory = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/notebooks/Tariq/final/"

# Combine the directory and filename to create the full file path
model_path = os.path.join(save_directory, model_filename)

joblib.dump(svm_classifier, model_path)

print(f"Model saved as {model_path}")

# Loading and testing

In [ ]:
import joblib
import numpy as np
import os
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the directory and filename where the model is saved
model_directory = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/notebooks/Tariq/final/"
model_filename = 'svm_model.pkl'

# Load the trained logistic regression model
model_path = os.path.join(model_directory, model_filename)
svm_classifier2 = joblib.load(model_path)

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

test_predictions = svm_classifier2.predict(test_features)

# Evaluate the model on the test set
test_accuracy = accuracy_score(test_y, test_predictions)
test_classification_report = classification_report(test_y, test_predictions, target_names=['0', '1', '2'], output_dict=True)

# Convert the classification report dictionary to a DataFrame
report_df = pd.DataFrame.from_dict(test_classification_report).T

# Format the DataFrame to display percentages
report_df['support'] = report_df['support'].astype(int)
report_df['accuracy'] = test_accuracy
report_df['accuracy'] = report_df['accuracy'].apply(lambda x: f'{x * 100:.2f}%')
report_df.iloc[:-1, :-1] = (report_df.iloc[:-1, :-1] * 100).applymap(lambda x: f'{x:.2f}%')

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print("Test Classification Report:\n", report_df)
